# Avance 5 Modelo Final 

## **Equipo #5**

**Integrantes y matriculas**

* Isaac Manriquez Calderon - A01206674
* Marcos Eduardo Garcia Ortiz - A01276213
* Jesus David Talamantes Morales - A01706335


In [6]:
import warnings
warnings.filterwarnings('ignore')

# Comando para recargar cambios en librerias automaticamente.
%load_ext autoreload 
%autoreload 2

# Librerias para el manejo del ambiente y documentos
import sys, os
from pathlib import Path
sys.path.append(os.path.abspath(os.path.join('..')))


import cv2
import numpy as np
from pathlib import Path
import json

# Librerias Locales del Repositorio.
from watermark.features import Encription_images
from watermark.dataset import DataHandler

# Librerias de Procesamiento de Imagenes
import pywt  # Wavelet transform
from scipy.fftpack import dct
from imagehash import phash  # Librería para hash perceptual

# Librerias de Machine Learning
from PIL import Image
import torch
from torchvision import models

2025-02-23 13:45:39.388 | INFO     | watermark.config:<module>:11 - PROJ_ROOT path is: C:\Users\55144\OneDrive - ArcelorMittal\Desktop\Trabajos MNA\TrimEne-Arp25\Proyecto integrador\watermarking


In [2]:
# Definir directorios
original_dir = Path("DataSet")  # Carpeta con imágenes originales
altered_dir = Path("altered_traces")  # Carpeta para imágenes alteradas

# Crear directorios si no existen
altered_dir.mkdir(parents=True, exist_ok=True)

# Transformaciones a aplicar
def apply_transformations(image):
    transformations = {}
    
    # 1. Compresión JPEG
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 50]  # Calidad 50
    _, compressed = cv2.imencode(".jpg", image, encode_param)
    transformations["compressed"] = cv2.imdecode(compressed, cv2.IMREAD_COLOR)
    
    # 2. Agregar Ruido Gaussiano
    noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    transformations["noisy"] = noisy_image
    
    # 3. Recorte
    h, w = image.shape[:2]
    cropped = image[int(0.1*h):int(0.9*h), int(0.1*w):int(0.9*w)]  # Recorte central
    transformations["cropped"] = cv2.resize(cropped, (w, h))  # Reescalar al tamaño original
    
    # 4. Rotación 15 grados
    M = cv2.getRotationMatrix2D((w//2, h//2), 15, 1)
    rotated = cv2.warpAffine(image, M, (w, h))
    transformations["rotated"] = rotated

    return transformations



In [3]:
# Procesar imágenes originales
for img_path in original_dir.glob("*.png"):
    img = cv2.imread(str(img_path))
    
    # Guardar original
    cv2.imwrite(str(altered_dir / f"{img_path.stem}_original.png"), img)
    
    # Aplicar transformaciones y guardar
    altered_images = apply_transformations(img)
    for name, altered_img in altered_images.items():
        cv2.imwrite(str(altered_dir / f"{img_path.stem}_{name}.png"), altered_img)

print(f"Se generaron {len(list(altered_dir.glob('*.png')))} imágenes alteradas.")


Se generaron 4510 imágenes alteradas.


## BER & NCC metrics 

In [7]:
# Cargar el modelo ResNet101 preentrenado
resnet101 = models.resnet101(pretrained=True)

# Remover la capa final de clasificación (la capa fc)
resnet101 = torch.nn.Sequential(*list(resnet101.children())[:-1])

resnet101.eval()  # Poner el modelo en modo evaluación

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [8]:
dh=DataHandler()

In [11]:
# Definir la ruta del directorio donde están las imágenes
directorio_imagenes = "altered_traces"

# Lista para almacenar las imágenes preprocesadas
imagenes_preprocesadas = []

# Obtener la lista de archivos de imagen filtrando por extensión
archivos_imagen = [archivo for archivo in os.listdir(directorio_imagenes)
                   if archivo.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Recorrer cada archivo y procesar la imagen
for archivo in archivos_imagen:
    ruta_completa = os.path.join(directorio_imagenes, archivo)
    # Abrir la imagen (convertirla a RGB si es necesario)
    img = Image.open(ruta_completa).convert('RGB')
    
    # Preprocesar la imagen para ResNet101 (usa tu función de preprocesamiento)
    img_tensor = dh.preprocesar_imagen_tensor(img)
    
    # Añadir la imagen preprocesada a la lista
    imagenes_preprocesadas.append(img_tensor)

# Ver el tamaño de las imágenes preprocesadas
print(f"Cantidad de imágenes preprocesadas: {len(imagenes_preprocesadas)}")

Cantidad de imágenes preprocesadas: 4510


In [12]:
# Lista para almacenar los vectores de características
feature_vectors = []

# Desactivar el cálculo de gradientes para la inferencia
with torch.no_grad():
    # Iterar sobre todas las imágenes preprocesadas
    for img_tensor in imagenes_preprocesadas:        
        # Pasar la imagen a través del modelo para obtener las características
        feature_map = resnet101(img_tensor)  # Salida de las capas convolucionales y pooling
        
        # Aplanar el tensor de características para obtener el vector de características
        feature_vector = feature_map.view(feature_map.size(0), -1)  # Aplanar a un vector de 2048 dimensiones
        
        # Añadir el vector de características a la lista
        feature_vectors.append(feature_vector)

print(f"Deep feature de la imagen 1: {feature_vectors[0]}")

Deep feature de la imagen 1: tensor([[0.4640, 0.4581, 0.0036,  ..., 2.3492, 0.0000, 0.2696]])


In [13]:
# Lista para almacenar los vectores de características con DWT, DCT y hash perceptual
feature_vectors_dwt_dct_hash = []

# Desactivar el cálculo de gradientes para la inferencia
with torch.no_grad():
    # Iterar sobre cada vector de características (tienes 902 vectores)
    for idx, feature_vector in enumerate(feature_vectors):        
        # Aplicar DWT sobre el vector de características (usando una transformación 1D)
        coeffs = pywt.dwt(feature_vector.squeeze().cpu().numpy(), 'db1')  # 'db1' es un tipo de wavelet común (Daubechies)
        cA, cD = coeffs  # Coeficientes aproximados (cA) y detalles (cD)
        
        # Aplicar DCT sobre los coeficientes aproximados (cA)
        cA_dct = dct(cA, norm='ortho')
        
        # Extraer los primeros 64 coeficientes válidos de la DCT
        low_freq_coeffs = cA_dct[:64].reshape(8,8)  # Tomamos los primeros 32 coeficientes
        
        # Normalizar los coeficientes para que estén en [0, 255] y convertir a uint8
        low_freq_matrix_norm = np.uint8(np.interp(low_freq_coeffs, 
                                                  (low_freq_coeffs.min(), low_freq_coeffs.max()), 
                                                  (0, 255)))

        # Convertir la matriz en imagen PIL
        img = Image.fromarray(low_freq_matrix_norm, mode="L")  

        # Aplicar el perceptual hash con un tamaño mayor (32x32 bits)
        ph = phash(img, hash_size=32)  

        # Convertir el hash en una matriz 32x32
        hash_matrix = np.array(ph.hash, dtype=int)

        # Guardar el hash en formato de matriz
        feature_vectors_dwt_dct_hash.append(hash_matrix)